In [1]:
import os

import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
import random
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
num_latent = 100

def set_seed(seed=2021):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [3]:
import torch

# 清理 GPU 缓存
torch.cuda.empty_cache()

In [9]:
# load data and create a label
loaded_mem_tensor = torch.load('mia/output_member_100_seqs.pt')
print(loaded_mem_tensor.shape)
mem_y = [1] * len(loaded_mem_tensor)  # 1 represent member class

loaded_nonmem_tensor = torch.load('mia/output_nonmember_100_seqs.pt')
print(loaded_nonmem_tensor.shape)
nonmem_y = [0] * len(loaded_nonmem_tensor)  # 0 represent non-member class
# loaded__mem_tensor.shape, loaded__nonmem_tensor.shape

torch.Size([100, 1000])
torch.Size([100, 1000])


In [10]:
print(len(loaded_mem_tensor))

100


In [141]:
type(loaded__mem_tensor), type(loaded_nonmem_tensor)

(torch.Tensor, torch.Tensor)

In [142]:
# device = torch.device('cuda:0')
# loaded__mem_tensor = loaded__mem_tensor.to(device)
# loaded__nonmem_tensor = loaded__nonmem_tensor.to(device)

In [143]:
# print(loaded__mem_tensor.device)  # 输出 mem 数据所在设备
# print(loaded__nonmem_tensor.device)  # 输出 nonmem 数据所在设备

cuda:0
cuda:0


In [151]:
# device = torch.device('cpu')  # 使用 CPU
# loaded__mem_tensor = loaded__mem_tensor.to(device)
# loaded__nonmem_tensor = loaded__nonmem_tensor.to(device)
# loaded__mem_tensor.shape, loaded__nonmem_tensor.shape

(torch.Size([10, 52740]), torch.Size([10, 52740]))

In [12]:
# Combine the data for training
data = torch.cat((loaded_mem_tensor, loaded_nonmem_tensor), dim=0)
labels = torch.tensor(mem_y + nonmem_y)
len(data), labels

(200,
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]))

In [13]:
# Split the dataset into training and testing

# 转换为 NumPy 格式以便划分
data_np = data.cpu().detach().numpy()
labels_np = labels.numpy()

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(data_np, labels_np, test_size=0.2, random_state=42)

In [14]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [15]:
# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [16]:
input_size = X_train.shape[1]
input_size

1000

In [17]:
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

In [18]:
mlp = MLP(input_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mlp.to(device)

MLP(
  (fc1): Linear(in_features=1000, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(mlp.parameters(), lr=0.01, momentum=0.7)

# accuarcy
def AccuarcyCompute(pred, label):
    pred = pred.cpu().data.numpy()
    label = label.cpu().data.numpy()
    #    print(pred.shape(),label.shape())
    test_np = (np.argmax(pred, 1) == label)
    test_np = np.float32(test_np)
    return np.mean(test_np)


losses = []
acces = []
eval_losses = []
eval_acces = []

for x in range(50):
    train_loss = 0
    train_acc = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = mlp(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()

        # _, pred = outputs.max(1)
        # print(outputs.shape)
        # _, predicted = torch.max(outputs, 1)
        # print(predicted.shape)
        # if(predicted == labels.cpu().numpy()[0]): # int(pred)
            # train_acc += 1
        train_acc += AccuarcyCompute(outputs, labels)

        # print(x, ":", AccuarcyCompute(outputs, labels))
    losses.append(train_loss / len(train_loader))
    acces.append(train_acc / len(train_loader))
    print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}'.format(x, train_loss / len(train_loader),
                                                                    train_acc / len(train_loader)))

acc_ans = 0
TruePositive = 0
FalsePositive = 0
TrueNegative = 0
FalseNegative = 0
for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = mlp(inputs)
    # print(outputs)
    # _, pred = outputs.max(1)
    # if int(pred) == labels.numpy()[0]:
    #     acc_ans += 1
    #     if int(pred) == 1:
    #         TruePositive = TruePositive + 1
    #     else:
    #         TrueNegative = TrueNegative + 1
    # else:
    #     if int(pred) == 1:
    #         FalsePositive = FalsePositive + 1
    #     else:
    #         FalseNegative = FalseNegative + 1
    _, predicted = torch.max(outputs, dim=1)
    for pred, label in zip(predicted, labels):
            if pred == label:
                acc_ans += 1
                if pred == 1:
                    TruePositive += 1
                else:
                    TrueNegative += 1
            else:
                if pred == 1:
                    FalsePositive += 1
                else:
                    FalseNegative += 1

print('TruePositive:', TruePositive)
print('FalsePositive:', FalsePositive)
print('TrueNegative:', TrueNegative)
print('FalseNegative', FalseNegative)
print("accuarcy: ")
print((acc_ans / len(test_loader)))

if (TruePositive + FalsePositive) != 0:
    print("precsion: ")
    print((TruePositive / (TruePositive + FalsePositive)))
if (TruePositive + FalseNegative) != 0:
    print("recall: ")
    print((TruePositive / (TruePositive + FalseNegative)))

if (TruePositive + FalseNegative) !=0:
    TPRate = TruePositive / (TruePositive + FalseNegative)
if (FalsePositive + TrueNegative) !=0:
    FPRate = FalsePositive / (FalsePositive + TrueNegative)
area = 0.5 * TPRate * FPRate + 0.5 * (TPRate + 1) * (1 - FPRate)
print("AUC: ")
print(area)

epoch: 0, Train Loss: 0.693149, Train Acc: 0.506250
epoch: 1, Train Loss: 0.693064, Train Acc: 0.506250
epoch: 2, Train Loss: 0.693087, Train Acc: 0.506250
epoch: 3, Train Loss: 0.693106, Train Acc: 0.506250
epoch: 4, Train Loss: 0.693072, Train Acc: 0.506250
epoch: 5, Train Loss: 0.693121, Train Acc: 0.506250
epoch: 6, Train Loss: 0.693093, Train Acc: 0.506250
epoch: 7, Train Loss: 0.693114, Train Acc: 0.506250
epoch: 8, Train Loss: 0.693105, Train Acc: 0.506250
epoch: 9, Train Loss: 0.693122, Train Acc: 0.506250
epoch: 10, Train Loss: 0.693099, Train Acc: 0.506250
epoch: 11, Train Loss: 0.693069, Train Acc: 0.506250
epoch: 12, Train Loss: 0.693133, Train Acc: 0.506250
epoch: 13, Train Loss: 0.693143, Train Acc: 0.506250
epoch: 14, Train Loss: 0.693077, Train Acc: 0.506250
epoch: 15, Train Loss: 0.693081, Train Acc: 0.506250
epoch: 16, Train Loss: 0.693118, Train Acc: 0.506250
epoch: 17, Train Loss: 0.693072, Train Acc: 0.506250
epoch: 18, Train Loss: 0.693092, Train Acc: 0.506250
epo